In [1]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
import numpy as np
import time




##### Load and inspect dataset

In [2]:
weather_detail_df = pd.read_csv('../Datasets/Original Files/Weather_Detail.csv')
weather_detail_df.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,DAPR,MDPR,PGTM,...,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09
0,US1KYJF0010,"PLEASURE RIDGE PARK 0.6 NNW, KY US",38.161724,-85.856597,139.9,1/1/2015,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US1KYBT0002,"MOUNT WASHINGTON 0.6 NNE, KY US",38.055096,-85.544697,208.2,1/1/2015,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US1KYBT0001,"FOX CHASE 1.4 W, KY US",38.044996,-85.715500,152.1,1/1/2015,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00013810,"LOUISVILLE BOWMAN FIELD, KY US",38.230210,-85.662940,164.6,1/1/2015,8.05,NaN,NaN,NaN,...,19.9,23.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US1KYJF0021,"VALLEY STATION 1.6 NNE, KY US",38.124900,-85.848780,158.5,1/1/2015,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
weather_detail_df.shape

(78129, 33)

In [ ]:
weather_detail_df.columns

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND',
       'DAPR', 'MDPR', 'PGTM', 'PRCP', 'PSUN', 'SNOW', 'SNWD', 'TAVG', 'TMAX',
       'TMIN', 'TOBS', 'TSUN', 'WDF2', 'WDF5', 'WESD', 'WESF', 'WSF2', 'WSF5',
       'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09'],
      dtype='object')

In [38]:
# Drop columns DAPR MDPR PGTM PSUN TOBS TSUN WESD WESF WT01 WT02 WT03 WT04 WT05 WT06 WT08 WT09

weather_detail_df.drop(['DAPR','MDPR','PGTM','PSUN','TAVG','TOBS','TSUN','WESD','WESF','WT01','WT02','WT03','WT04','WT05','WT06','WT08','WT09'], axis = 1, inplace =True)


In [39]:
# making sure columns dropped

weather_detail_df

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
0,US1KYJF0010,"PLEASURE RIDGE PARK 0.6 NNW, KY US",38.161724,-85.856597,139.9,1/1/2015,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,US1KYBT0002,"MOUNT WASHINGTON 0.6 NNE, KY US",38.055096,-85.544697,208.2,1/1/2015,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US1KYBT0001,"FOX CHASE 1.4 W, KY US",38.044996,-85.715500,152.1,1/1/2015,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00013810,"LOUISVILLE BOWMAN FIELD, KY US",38.230210,-85.662940,164.6,1/1/2015,8.05,0.0,NaN,NaN,40.0,22.0,230.0,250.0,19.9,23.9
4,US1KYJF0021,"VALLEY STATION 1.6 NNE, KY US",38.124900,-85.848780,158.5,1/1/2015,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78124,US1KYJF0001,"ANCHORAGE 2.8 NE, KY US",38.295967,-85.505339,231.0,10/1/2025,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78125,US1KYJF0002,"OLD BROWNSBORO PLACE 0.3 SW, KY US",38.285860,-85.614080,187.5,10/1/2025,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78126,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",38.177380,-85.730770,146.3,10/1/2025,NaN,0.0,0.0,0.0,85.0,62.0,NaN,NaN,NaN,NaN
78127,US1KYJF0087,"LOUISVILLE 6.0 E, KY US",38.262200,-85.648400,166.1,10/1/2025,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Review Data types, look for nulls, and duplicates

In [40]:
weather_detail_df.isnull().sum()

STATION          0
NAME             0
LATITUDE         0
LONGITUDE        0
ELEVATION        0
DATE             0
AWND         70364
PRCP          1719
SNOW         33963
SNWD         56655
TMAX         60406
TMIN         60448
WDF2         70357
WDF5         70365
WSF2         70357
WSF5         70364
dtype: int64

In [41]:
#dropping rows that have nulls in the TMAX column - Since TMAX is essential for the analysis, rows without a recorded max temperature aren't useful

weather_detail_df.dropna(subset=['TMAX'], inplace=True)



In [42]:
# resetting index after drop 

weather_detail_df = weather_detail_df.reset_index(drop=True)

In [43]:
weather_detail_df.isnull().sum()

STATION         0
NAME            0
LATITUDE        0
LONGITUDE       0
ELEVATION       0
DATE            0
AWND         9958
PRCP           39
SNOW         9932
SNWD         8088
TMAX            0
TMIN           42
WDF2         9951
WDF5         9959
WSF2         9951
WSF5         9958
dtype: int64

In [44]:
#Replacing remaing nulls with zero since it's likely that there was no data to record for that day 

weather_detail_df.fillna(0,inplace=True)  

In [45]:
weather_detail_df.isnull().sum() #checking to make sure there are no nulls

STATION      0
NAME         0
LATITUDE     0
LONGITUDE    0
ELEVATION    0
DATE         0
AWND         0
PRCP         0
SNOW         0
SNWD         0
TMAX         0
TMIN         0
WDF2         0
WDF5         0
WSF2         0
WSF5         0
dtype: int64

In [62]:
duplicates = weather_detail_df.duplicated()
weather_detail_df[duplicates]

,STATION,NAME,BEGIN_LAT,BEGIN_LON,ELEVATION,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,Zip


In [47]:
weather_detail_df.dtypes

STATION       object
NAME          object
LATITUDE     float64
LONGITUDE    float64
ELEVATION    float64
DATE          object
AWND         float64
PRCP         float64
SNOW         float64
SNWD         float64
TMAX         float64
TMIN         float64
WDF2         float64
WDF5         float64
WSF2         float64
WSF5         float64
dtype: object

In [48]:
weather_detail_df = weather_detail_df.rename(columns={"LATITUDE": "BEGIN_LAT", "LONGITUDE": "BEGIN_LON"})
weather_detail_df

,STATION,NAME,BEGIN_LAT,BEGIN_LON,ELEVATION,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5
0,USW00013810,"LOUISVILLE BOWMAN FIELD, KY US",38.23021,-85.66294,164.6,1/1/2015,8.05,0.00,0.0,0.0,40.0,22.0,230.0,250.0,19.9,23.9
1,USC00154955,"LOUISVILLE MCALPINE, KY US",38.27780,-85.79180,134.1,1/1/2015,0.00,0.00,0.0,0.0,28.0,20.0,0.0,0.0,0.0,0.0
2,USC00150630,"BERNHEIM FOREST, KY US",37.91610,-85.65720,167.6,1/1/2015,0.00,0.02,0.0,0.0,41.0,16.0,0.0,0.0,0.0,0.0
3,USC00154958,"LOUISVILLE WEATHER FORECAST OFFICE, KY US",38.11510,-85.64450,192.6,1/1/2015,0.00,0.00,0.0,0.0,40.0,19.0,0.0,0.0,0.0,0.0
4,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",38.17738,-85.73077,146.3,1/1/2015,9.17,0.00,0.0,0.0,41.0,23.0,230.0,240.0,21.0,25.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17718,USC00154955,"LOUISVILLE MCALPINE, KY US",38.27780,-85.79180,134.1,9/30/2025,0.00,0.00,0.0,0.0,87.0,63.0,0.0,0.0,0.0,0.0
17719,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",38.17738,-85.73077,146.3,9/30/2025,0.00,0.00,0.0,0.0,87.0,63.0,0.0,0.0,0.0,0.0
17720,USW00013810,"LOUISVILLE BOWMAN FIELD, KY US",38.23021,-85.66294,164.6,10/1/2025,0.00,0.00,0.0,0.0,86.0,57.0,0.0,0.0,0.0,0.0
17721,USC00154955,"LOUISVILLE MCALPINE, KY US",38.27780,-85.79180,134.1,10/1/2025,0.00,0.00,0.0,0.0,87.0,64.0,0.0,0.0,0.0,0.0


##### Reverse Geocoding to return zipcode using latitude and longitude

In [49]:
# There were too many rows to loop through like I did for the tornado tracks.  So I found the unique coordinates, put those in a dictionary, and looped through those to find the zipcodes 

unique_coords = weather_detail_df[['BEGIN_LAT','BEGIN_LON']].drop_duplicates()

unique_coords = unique_coords.reset_index(drop=True) #reset index 

unique_coords




,BEGIN_LAT,BEGIN_LON
0,38.23021,-85.66294
1,38.27780,-85.79180
2,37.91610,-85.65720
3,38.11510,-85.64450
4,38.17738,-85.73077


In [51]:
geolocator = Nominatim(user_agent="tornado_project")

# Dictionary to store results
coord_to_zip = {}

for i in range(len(unique_coords)):
    try:
        lat = unique_coords.loc[i, 'BEGIN_LAT']
        lon = unique_coords.loc[i, 'BEGIN_LON']
        location = geolocator.reverse(f"{lat}, {lon}", timeout=10)
        address = location.raw.get('address', {})
        zipcode = address.get('postcode', np.nan)

        # Save into dictionary
        coord_to_zip[(lat, lon)] = zipcode

        # Store in DataFrame
        unique_coords.loc[i, 'Zip'] = zipcode

    except Exception as e:
        print(f"Error at row {i}: {e}")
        coord_to_zip[(lat, lon)] = np.nan
        unique_coords.loc[i, 'Zip'] = np.nan

    time.sleep(1)  # respect Nominatim rate limits






In [54]:
# to merge the unique coords and weather detail df 

weather_detail_df = weather_detail_df.merge(unique_coords, on=['BEGIN_LAT','BEGIN_LON'], how='left')
weather_detail_df

,STATION,NAME,BEGIN_LAT,BEGIN_LON,ELEVATION,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,Zip_x,Zip_y
0,USW00013810,"LOUISVILLE BOWMAN FIELD, KY US",38.23021,-85.66294,164.6,1/1/2015,8.05,0.00,0.0,0.0,40.0,22.0,230.0,250.0,19.9,23.9,40041,40041
1,USC00154955,"LOUISVILLE MCALPINE, KY US",38.27780,-85.79180,134.1,1/1/2015,0.00,0.00,0.0,0.0,28.0,20.0,0.0,0.0,0.0,0.0,40212,40212
2,USC00150630,"BERNHEIM FOREST, KY US",37.91610,-85.65720,167.6,1/1/2015,0.00,0.02,0.0,0.0,41.0,16.0,0.0,0.0,0.0,0.0,40110,40110
3,USC00154958,"LOUISVILLE WEATHER FORECAST OFFICE, KY US",38.11510,-85.64450,192.6,1/1/2015,0.00,0.00,0.0,0.0,40.0,19.0,0.0,0.0,0.0,0.0,40229,40229
4,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",38.17738,-85.73077,146.3,1/1/2015,9.17,0.00,0.0,0.0,41.0,23.0,230.0,240.0,21.0,25.1,40209,40209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17718,USC00154955,"LOUISVILLE MCALPINE, KY US",38.27780,-85.79180,134.1,9/30/2025,0.00,0.00,0.0,0.0,87.0,63.0,0.0,0.0,0.0,0.0,40212,40212
17719,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",38.17738,-85.73077,146.3,9/30/2025,0.00,0.00,0.0,0.0,87.0,63.0,0.0,0.0,0.0,0.0,40209,40209
17720,USW00013810,"LOUISVILLE BOWMAN FIELD, KY US",38.23021,-85.66294,164.6,10/1/2025,0.00,0.00,0.0,0.0,86.0,57.0,0.0,0.0,0.0,0.0,40041,40041
17721,USC00154955,"LOUISVILLE MCALPINE, KY US",38.27780,-85.79180,134.1,10/1/2025,0.00,0.00,0.0,0.0,87.0,64.0,0.0,0.0,0.0,0.0,40212,40212


In [ ]:
weather_detail_df = weather_detail_df.drop(columns=['Zip_y'])



In [59]:
weather_detail_df = weather_detail_df.rename(columns={'Zip_x': 'Zip'})
weather_detail_df

,STATION,NAME,BEGIN_LAT,BEGIN_LON,ELEVATION,DATE,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,Zip
0,USW00013810,"LOUISVILLE BOWMAN FIELD, KY US",38.23021,-85.66294,164.6,1/1/2015,8.05,0.00,0.0,0.0,40.0,22.0,230.0,250.0,19.9,23.9,40041
1,USC00154955,"LOUISVILLE MCALPINE, KY US",38.27780,-85.79180,134.1,1/1/2015,0.00,0.00,0.0,0.0,28.0,20.0,0.0,0.0,0.0,0.0,40212
2,USC00150630,"BERNHEIM FOREST, KY US",37.91610,-85.65720,167.6,1/1/2015,0.00,0.02,0.0,0.0,41.0,16.0,0.0,0.0,0.0,0.0,40110
3,USC00154958,"LOUISVILLE WEATHER FORECAST OFFICE, KY US",38.11510,-85.64450,192.6,1/1/2015,0.00,0.00,0.0,0.0,40.0,19.0,0.0,0.0,0.0,0.0,40229
4,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",38.17738,-85.73077,146.3,1/1/2015,9.17,0.00,0.0,0.0,41.0,23.0,230.0,240.0,21.0,25.1,40209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17718,USC00154955,"LOUISVILLE MCALPINE, KY US",38.27780,-85.79180,134.1,9/30/2025,0.00,0.00,0.0,0.0,87.0,63.0,0.0,0.0,0.0,0.0,40212
17719,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",38.17738,-85.73077,146.3,9/30/2025,0.00,0.00,0.0,0.0,87.0,63.0,0.0,0.0,0.0,0.0,40209
17720,USW00013810,"LOUISVILLE BOWMAN FIELD, KY US",38.23021,-85.66294,164.6,10/1/2025,0.00,0.00,0.0,0.0,86.0,57.0,0.0,0.0,0.0,0.0,40041
17721,USC00154955,"LOUISVILLE MCALPINE, KY US",38.27780,-85.79180,134.1,10/1/2025,0.00,0.00,0.0,0.0,87.0,64.0,0.0,0.0,0.0,0.0,40212


In [ ]:
weather_detail_df

##### Save DF into new CSV

In [60]:
weather_detail_df.to_csv("weather_detail_cleaned.csv", index=False)